In [1]:
import os
import itertools
import logging
logging.basicConfig()
import json
import tempfile
from copy import deepcopy
from collections import defaultdict
from functools import partial

import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import GridspecLayout, Layout, Label, VBox, HBox, Box, Output, Button, SelectMultiple, Checkbox, IntText, Textarea

In [2]:
logging.getLogger("cicliminds_lib").setLevel(logging.INFO)
from cicliminds_lib.query.files import get_datasets

In [3]:
from cicliminds.widgets.filter import FilterWidget
from cicliminds.widgets.filtered import FilteredWidget
from cicliminds.widgets.staging import StagingWidget

from cicliminds.interface import expand_state_into_queries
from cicliminds.backend import write_dataset_by_query
from cicliminds.backend import plot_by_query

In [4]:
plt.rcParams["figure.figsize"] = (12,8)
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 8
plt.rcParams["xtick.major.width"] = 1.6
plt.rcParams["xtick.minor.width"] = 0.8
plt.rcParams["xtick.minor.size"] = 4
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.width"] = 1.6
plt.rcParams["ytick.minor.width"] = 0.8
plt.rcParams["ytick.major.size"] = 8
plt.rcParams["ytick.minor.size"] = 4
plt.rcParams["font.size"] = 16
plt.rcParams["lines.linewidth"] = 3
plt.rcParams["lines.markersize"] = 5
plt.rcParams["savefig.dpi"] = 300/2.4
plt.rcParams["savefig.transparent"] = False
plt.rcParams["savefig.facecolor"] = "white"

In [5]:
DATA_DIR = os.environ["DATA_DIR"]
display(DATA_DIR)

'/home/viktoana/projects/cicero/data/Climdex_base1981-2010'

In [6]:
DATASETS = get_datasets(DATA_DIR)

In [7]:
DATASETS

,variable,frequency,model,scenario,init_params,timespan
path,,,,,,
/home/viktoana/projects/cicero/data/Climdex_base1981-2010/altcddETCCDI_yr_CNRM-ESM2-1_historical_r1i1p1f2_1850-2014.nc,altcddETCCDI,yr,CNRM-ESM2-1,historical,r1i1p1f2,1850-2014
/home/viktoana/projects/cicero/data/Climdex_base1981-2010/altcddETCCDI_yr_ACCESS-CM2_historical_r1i1p1f1_1850-2014.nc,altcddETCCDI,yr,ACCESS-CM2,historical,r1i1p1f1,1850-2014
/home/viktoana/projects/cicero/data/Climdex_base1981-2010/altcddETCCDI_yr_CNRM-ESM2-1_ssp126_r1i1p1f2_2015-2100.nc,altcddETCCDI,yr,CNRM-ESM2-1,ssp126,r1i1p1f2,2015-2100
/home/viktoana/projects/cicero/data/Climdex_base1981-2010/altcddETCCDI_yr_ACCESS-CM2_ssp126_r1i1p1f1_2015-2100.nc,altcddETCCDI,yr,ACCESS-CM2,ssp126,r1i1p1f1,2015-2100
/home/viktoana/projects/cicero/data/Climdex_base1981-2010/altcddETCCDI_yr_CNRM-ESM2-1_ssp245_r1i1p1f2_2015-2100.nc,altcddETCCDI,yr,CNRM-ESM2-1,ssp245,r1i1p1f2,2015-2100
...,...,...,...,...,...,...
/home/viktoana/projects/cicero/data/Climdex_base1981-2010/wsdiETCCDI_yr_UKESM1-0-LL_historical_r1i1p1f2_1850-2014.nc,wsdiETCCDI,yr,UKESM1-0-LL,historical,r1i1p1f2,1850-2014
/home/viktoana/projects/cicero/data/Climdex_base1981-2010/wsdiETCCDI_yr_UKESM1-0-LL_ssp126_r1i1p1f2_2015-2100.nc,wsdiETCCDI,yr,UKESM1-0-LL,ssp126,r1i1p1f2,2015-2100
/home/viktoana/projects/cicero/data/Climdex_base1981-2010/wsdiETCCDI_yr_UKESM1-0-LL_ssp245_r1i1p1f2_2015-2100.nc,wsdiETCCDI,yr,UKESM1-0-LL,ssp245,r1i1p1f2,2015-2100


In [8]:
def update_filters(filters_widget, change):
    global filtered_widget
    filtered_dataset = filters_widget.get_filtered_dataset()
    filters_widget.update_state_from_dataset(filtered_dataset)
    if filtered_dataset.shape[0] > 200:
        return
    filtered_widget.update_state_from_dataset(filtered_dataset)

In [9]:
def rebuild_one_block_action(block, change):
    cfg = json.loads(block.children[0].children[0].value)
    output = block.children[1]
    fig, ax = plt.subplots()
    with tempfile.NamedTemporaryFile("r") as dataset:
        write_dataset_by_query(DATASETS, cfg, dataset.name)
        plot_by_query(ax, dataset, cfg)
    plt.close()
    with output:
        output.clear_output()
        display(fig)

In [10]:
def staged_block(model_config):
    config_widget = Textarea(value=json.dumps(model_config, indent=True), layout={"flex": "6 1 260px", "height": "10em", "overflow": "hidden", "margin": "0 20px 0 0"})
    unstage_button = Button(description="Unstage", button_style="danger", icon="trash")
    rebuild_button = Button(description="Rebuild", button_style="success", icon="redo")
    output = Output(layout={"flex": "1 1 0px"})
    block = VBox([
        HBox([config_widget, VBox([unstage_button, rebuild_button], layout={"flex": "1 1 100px", "margin": "0 20px 0 0"})]),
        output
    ], layout={"margin": "5px 0"})
    rebuild_button.on_click(partial(rebuild_one_block_action, block))
    unstage_button.on_click(unstage_one_block_action)
    return block

In [11]:
def stage_action(staging_widget, change):
    global filtered_widget
    global staged_list
    to_agg = filtered_widget.get_selected_dataset()
    agg_params = staging_widget.get_state()
    to_add = []
    for model_config in expand_state_into_queries(to_agg, agg_params):
        to_add.append(staged_block(model_config))
    staged_list.children = tuple(to_add) + staged_list.children

In [12]:
def rebuild_all_action(change):
    global staged_list
    for block in staged_list.children:
        block.children[0].children[1].children[1].click()
        
def unstage_all_action(change):
    global staged_list
    staged_list.children = tuple()
    
def unstage_one_block_action(change):
    global staged_list
    for i, block in enumerate(staged_list.children):
        if change == block.children[0].children[1].children[0]:
            break
    staged_list.children = staged_list.children[:i] + staged_list.children[i+1:]

In [13]:
filter_widget = FilterWidget(DATASETS)
filter_widget.observe(update_filters)
filtered_widget = FilteredWidget()
staging_widget = StagingWidget()
staging_widget.observe(stage_action)

# staged widget

button_rebuild_all = Button(description="Rebuild all", icon="redo", button_style="success")
button_rebuild_all.on_click(rebuild_all_action)
button_unstage_all = Button(description="Unstage all", icon="trash", button_style="danger")
button_unstage_all.on_click(unstage_all_action)
staged_controls = HBox([button_rebuild_all, button_unstage_all])
staged_list = VBox()

staged_widget = VBox([
    Label("Staged for plotting:"),
    staged_controls,
    staged_list])

app = VBox([filter_widget.render(),
            filtered_widget.render(),
            staging_widget.render(),
            staged_widget])

output = Output()
display(output, app)
filter_widget.reset_filters()

Output()